In [7]:
%%writefile task1_omp.cpp

#include <iostream> // подключает cout
#include <vector> // подключает vector
#include <cstdlib> // подключает atoi
#include <cmath> // подключает sqrt и fabs
#include <omp.h> // подключает OpenMP
#include <algorithm> // подключает max

using namespace std; // чтобы не писать std::

static void fill_array(vector<double>& a) { // заполняет массив
    for (int i = 0; i < (int)a.size(); i++) { // цикл по массиву
        a[i] = (double)(rand() % 1000) / 10.0; // кладёт число 0..99.9
    }
}

static void calc_seq(const vector<double>& a, double& sum, double& sumsq) { // считает сумму и сумму квадратов последовательно
    sum = 0.0; // обнуляет сумму
    sumsq = 0.0; // обнуляет сумму квадратов
    for (int i = 0; i < (int)a.size(); i++) { // цикл по массиву
        double x = a[i]; // берёт элемент
        sum += x; // прибавляет в сумму
        sumsq += x * x; // прибавляет квадрат в сумму квадратов
    }
}

static void calc_omp(const vector<double>& a, double& sum, double& sumsq) { // считает сумму и сумму квадратов параллельно
    sum = 0.0; // обнуляет сумму
    sumsq = 0.0; // обнуляет сумму квадратов
#pragma omp parallel for reduction(+:sum,sumsq) // параллельный цикл с редукцией
    for (int i = 0; i < (int)a.size(); i++) { // цикл по массиву
        double x = a[i]; // берёт элемент
        sum += x; // добавляет в общую сумму
        sumsq += x * x; // добавляет в общую сумму квадратов
    }
}

static double avg_time_seq(const vector<double>& a, int reps, double& sum, double& sumsq) { // меряет среднее время seq
    double t = 0.0; // хранит суммарное время
    for (int r = 0; r < reps; r++) { // повторяет несколько раз
        double t0 = omp_get_wtime(); // старт времени
        calc_seq(a, sum, sumsq); // запускает seq расчёт
        double t1 = omp_get_wtime(); // конец времени
        t += (t1 - t0); // добавляет длительность
    }
    return t / (double)max(1, reps); // возвращает среднее время
}

static double avg_time_omp(const vector<double>& a, int reps, double& sum, double& sumsq) { // меряет среднее время omp
    double t = 0.0; // хранит суммарное время
    for (int r = 0; r < reps; r++) { // повторяет несколько раз
        double t0 = omp_get_wtime(); // старт времени
        calc_omp(a, sum, sumsq); // запускает omp расчёт
        double t1 = omp_get_wtime(); // конец времени
        t += (t1 - t0); // добавляет длительность
    }
    return t / (double)max(1, reps); // возвращает среднее время
}

static double clamp01(double x) { // ограничивает число от 0 до 1
    if (x < 0.0) return 0.0; // если ниже 0 то 0
    if (x > 1.0) return 1.0; // если выше 1 то 1
    return x; // иначе возвращает x
}

int main(int argc, char** argv) { // main
    int N = 10000000; // размер массива
    int threads = 0; // сколько потоков
    int reps = 5; // сколько повторов
    if (argc >= 2) N = atoi(argv[1]); // читает N из аргумента
    if (argc >= 3) threads = atoi(argv[2]); // читает threads из аргумента
    if (argc >= 4) reps = atoi(argv[3]); // читает reps из аргумента

    if (threads > 0) omp_set_num_threads(threads); // задаёт число потоков
    omp_set_dynamic(0); // запрещает OpenMP менять потоки сам

    srand(123); // фиксирует seed
    vector<double> a(N); // создаёт массив
    fill_array(a); // заполняет массив

    double warm_sum = 0.0; // переменная прогрева
    double warm_sumsq = 0.0; // переменная прогрева
    calc_seq(a, warm_sum, warm_sumsq); // прогрев seq
    calc_omp(a, warm_sum, warm_sumsq); // прогрев omp

    double seq_sum = 0.0; // сумма seq
    double seq_sumsq = 0.0; // сумма квадратов seq
    double omp_sum = 0.0; // сумма omp
    double omp_sumsq = 0.0; // сумма квадратов omp

    double t_seq = avg_time_seq(a, reps, seq_sum, seq_sumsq); // время seq
    double t_omp = avg_time_omp(a, reps, omp_sum, omp_sumsq); // время omp

    int used_threads = omp_get_max_threads(); // фактические потоки
    double speedup = (t_omp > 0 ? t_seq / t_omp : 0.0); // ускорение

    if (speedup > (double)used_threads) speedup = (double)used_threads; // защита от странных значений

    double p = (double)max(1, used_threads); // p для Амдала
    double s = (speedup > 1e-12 ? speedup : 1.0); // s для Амдала
    double denom = (1.0 - 1.0 / p); // знаменатель
    if (denom < 1e-12) denom = 1e-12; // защита
    double f = ((1.0 / s) - (1.0 / p)) / denom; // оценивает долю последовательной части f
    f = clamp01(f); // ограничивает f 0..1
    double parallel_part = 1.0 - f; // оценивает долю параллельной части

    double mean = omp_sum / (double)N; // считает среднее
    double var = (omp_sumsq / (double)N) - mean * mean; // считает дисперсию
    if (var < 0) var = 0; // защита от -0
    double stddev = sqrt(var); // стандартное отклонение

    double diff = fabs(seq_sum - omp_sum); // разница сумм
    double rel = diff / max(1.0, fabs(seq_sum)); // относительная ошибка
    bool ok_sum = (rel < 1e-9); // проверка корректности

    cout << "task 1\n"; // печатает метку
    cout << "n = " << N << "\n"; // печатает N
    cout << "threads = " << used_threads << "\n"; // печатает потоки
    cout << "seq ms = " << (t_seq * 1000.0) << "\n"; // печатает seq время
    cout << "omp ms = " << (t_omp * 1000.0) << "\n"; // печатает omp время
    cout << "speedup = " << speedup << "\n"; // печатает ускорение
    cout << "serial_part f = " << f << "\n"; // печатает последовательную долю
    cout << "parallel_part = " << parallel_part << "\n"; // печатает параллельную долю
    cout << "sum ok = " << (ok_sum ? "да" : "нет") << "\n"; // печатает проверку
    cout << "mean = " << mean << "\n"; // печатает mean
    cout << "var = " << var << "\n"; // печатает var
    cout << "std = " << stddev << "\n"; // печатает std
    return 0; // завершает программу
}


Overwriting task1_omp.cpp


In [8]:
%%bash
g++ -O2 -fopenmp task1_omp.cpp -o task1_omp

export OMP_PROC_BIND=true
export OMP_PLACES=cores
export OMP_DYNAMIC=false

./task1_omp 10000000 1 5 | tee out1.txt
./task1_omp 10000000 2 5 | tee out2.txt
./task1_omp 10000000 4 5 | tee out4.txt
./task1_omp 10000000 8 5 | tee out8.txt


task 1
n = 10000000
threads = 1
seq ms = 14.5836
omp ms = 14.9502
speedup = 0.975473
serial_part f = 1
parallel_part = 0
sum ok = да
mean = 49.9353
var = 833.348
std = 28.8678
task 1
n = 10000000
threads = 2
seq ms = 14.4841
omp ms = 8.19282
speedup = 1.7679
serial_part f = 0.131284
parallel_part = 0.868716
sum ok = да
mean = 49.9353
var = 833.348
std = 28.8678
task 1
n = 10000000
threads = 4
seq ms = 14.7065
omp ms = 9.86706
speedup = 1.49047
serial_part f = 0.561241
parallel_part = 0.438759
sum ok = да
mean = 49.9353
var = 833.348
std = 28.8678
task 1
n = 10000000
threads = 8
seq ms = 17.6666
omp ms = 15.0418
speedup = 1.17451
serial_part f = 0.830196
parallel_part = 0.169804
sum ok = да
mean = 49.9353
var = 833.348
std = 28.8678


In [9]:
%%writefile task2_cuda_memory.cu

#include <cuda_runtime.h> // подключает CUDA
#include <iostream> // подключает cout
#include <vector> // подключает vector
#include <cstdlib> // подключает atoi
#include <cmath> // подключает fabs

using namespace std; // чтобы не писать std::

static void cuda_ok(cudaError_t err, const char* msg) { // проверка CUDA
    if (err != cudaSuccess) { // если ошибка
        cout << "CUDA error (" << msg << "): " << cudaGetErrorString(err) << "\n"; // выводит ошибку
        exit(1); // завершает
    }
}

__global__ void k_coalesced(const float* in, float* out, int n) { // коалесцированный доступ
    int i = blockIdx.x * blockDim.x + threadIdx.x; // индекс
    if (i < n) out[i] = in[i] * 2.0f; // читает и пишет подряд
}

__device__ __forceinline__ int perm(int tid, int blockSize, int stride) { // перестановка индекса
    return (tid * stride) & (blockSize - 1); // даёт "прыгающий" индекс внутри блока
}

__global__ void k_uncoalesced(const float* in, float* out, int n, int stride) { // некоалесцированный доступ
    int base = blockIdx.x * blockDim.x; // база блока
    int tid = threadIdx.x; // tid
    int p = perm(tid, blockDim.x, stride); // переставляет tid
    int i = base + p; // делает прыгающий индекс
    if (i < n) out[i] = in[i] * 2.0f; // читает плохо по памяти
}

__global__ void k_shared_fix(const float* in, float* out, int n, int stride) { // оптимизация через shared
    extern __shared__ float sh[]; // shared память
    int base = blockIdx.x * blockDim.x; // база блока
    int tid = threadIdx.x; // tid

    int load_i = base + tid; // индекс чтения подряд
    float x = 0.0f; // значение
    if (load_i < n) x = in[load_i]; // читает коалесцированно
    sh[tid] = x; // кладёт в shared
    __syncthreads(); // синхронизация

    int p = perm(tid, blockDim.x, stride); // перестановка
    int out_i = base + p; // индекс записи
    if (out_i < n) out[out_i] = sh[p] * 2.0f; // берёт из shared и пишет
}

static float time_kernel_coal(const float* d_in, float* d_out, int n, int blocks, int threads, int iters) { // время coalesced
    cudaEvent_t s, e; // события
    cuda_ok(cudaEventCreate(&s), "event create"); // создаёт start
    cuda_ok(cudaEventCreate(&e), "event create"); // создаёт end
    k_coalesced<<<blocks, threads>>>(d_in, d_out, n); // прогрев
    cuda_ok(cudaDeviceSynchronize(), "sync"); // ждёт
    cuda_ok(cudaEventRecord(s), "record start"); // старт
    for (int i = 0; i < iters; i++) { // цикл итераций
        k_coalesced<<<blocks, threads>>>(d_in, d_out, n); // запускает ядро
    }
    cuda_ok(cudaEventRecord(e), "record end"); // конец
    cuda_ok(cudaEventSynchronize(e), "sync end"); // ждёт
    float ms = 0.0f; // ms
    cuda_ok(cudaEventElapsedTime(&ms, s, e), "elapsed"); // считает время
    cuda_ok(cudaEventDestroy(s), "destroy"); // удаляет
    cuda_ok(cudaEventDestroy(e), "destroy"); // удаляет
    return ms / (float)max(1, iters); // среднее
}

static float time_kernel_bad(const float* d_in, float* d_out, int n, int blocks, int threads, int iters, int stride) { // время uncoalesced
    cudaEvent_t s, e; // события
    cuda_ok(cudaEventCreate(&s), "event create"); // start
    cuda_ok(cudaEventCreate(&e), "event create"); // end
    k_uncoalesced<<<blocks, threads>>>(d_in, d_out, n, stride); // прогрев
    cuda_ok(cudaDeviceSynchronize(), "sync"); // ждёт
    cuda_ok(cudaEventRecord(s), "record start"); // старт
    for (int i = 0; i < iters; i++) { // цикл
        k_uncoalesced<<<blocks, threads>>>(d_in, d_out, n, stride); // запускает ядро
    }
    cuda_ok(cudaEventRecord(e), "record end"); // конец
    cuda_ok(cudaEventSynchronize(e), "sync end"); // ждёт
    float ms = 0.0f; // ms
    cuda_ok(cudaEventElapsedTime(&ms, s, e), "elapsed"); // время
    cuda_ok(cudaEventDestroy(s), "destroy"); // удаляет
    cuda_ok(cudaEventDestroy(e), "destroy"); // удаляет
    return ms / (float)max(1, iters); // среднее
}

static float time_kernel_shared(const float* d_in, float* d_out, int n, int blocks, int threads, int iters, int stride) { // время shared_fix
    cudaEvent_t s, e; // события
    cuda_ok(cudaEventCreate(&s), "event create"); // start
    cuda_ok(cudaEventCreate(&e), "event create"); // end
    k_shared_fix<<<blocks, threads, threads * (int)sizeof(float)>>>(d_in, d_out, n, stride); // прогрев
    cuda_ok(cudaDeviceSynchronize(), "sync"); // ждёт
    cuda_ok(cudaEventRecord(s), "record start"); // старт
    for (int i = 0; i < iters; i++) { // цикл
        k_shared_fix<<<blocks, threads, threads * (int)sizeof(float)>>>(d_in, d_out, n, stride); // запускает ядро
    }
    cuda_ok(cudaEventRecord(e), "record end"); // конец
    cuda_ok(cudaEventSynchronize(e), "sync end"); // ждёт
    float ms = 0.0f; // ms
    cuda_ok(cudaEventElapsedTime(&ms, s, e), "elapsed"); // время
    cuda_ok(cudaEventDestroy(s), "destroy"); // удаляет
    cuda_ok(cudaEventDestroy(e), "destroy"); // удаляет
    return ms / (float)max(1, iters); // среднее
}

int main(int argc, char** argv) { // main
    int N = 1 << 24; // размер
    int iters = 50; // итерации
    if (argc >= 2) N = atoi(argv[1]); // читает N
    if (argc >= 3) iters = atoi(argv[2]); // читает iters

    int threads = 256; // потоки
    int blocks = (N + threads - 1) / threads; // блоки
    int stride = 33; // stride для плохого доступа

    vector<float> h_in(N); // host массив
    for (int i = 0; i < N; i++) h_in[i] = (float)(i % 100); // заполняет

    float* d_in = nullptr; // device in
    float* d_out = nullptr; // device out
    cuda_ok(cudaMalloc(&d_in, N * (int)sizeof(float)), "malloc d_in"); // malloc
    cuda_ok(cudaMalloc(&d_out, N * (int)sizeof(float)), "malloc d_out"); // malloc
    cuda_ok(cudaMemcpy(d_in, h_in.data(), N * (int)sizeof(float), cudaMemcpyHostToDevice), "h2d"); // копия

    float ms_coal = time_kernel_coal(d_in, d_out, N, blocks, threads, iters); // время норм
    float ms_bad = time_kernel_bad(d_in, d_out, N, blocks, threads, iters, stride); // время плохое
    float ms_sh = time_kernel_shared(d_in, d_out, N, blocks, threads, iters, stride); // время shared

    cout << "task 2\n"; // метка
    cout << "n = " << N << "\n"; // n
    cout << "coalesced ms = " << ms_coal << "\n"; // вывод
    cout << "uncoalesced ms = " << ms_bad << "\n"; // вывод
    cout << "shared_fix ms = " << ms_sh << "\n"; // вывод

    cuda_ok(cudaFree(d_in), "free d_in"); // free
    cuda_ok(cudaFree(d_out), "free d_out"); // free
    return 0; // выход
}


Overwriting task2_cuda_memory.cu


In [10]:
%%bash
nvcc -O2 -gencode arch=compute_75,code=sm_75 task2_cuda_memory.cu -o task2_cuda_memory
./task2_cuda_memory 16777216 50


task 2
n = 16777216
coalesced ms = 0.56873
uncoalesced ms = 0.595256
shared_fix ms = 0.640637


In [12]:
%%writefile task3_hybrid.cu

#include <cuda_runtime.h> // подключает CUDA
#include <iostream> // подключает cout
#include <cstdlib> // подключает atoi
#include <omp.h> // подключает omp_get_wtime

using namespace std; // чтобы не писать std::

static void cuda_ok(cudaError_t err, const char* msg) { // проверка CUDA
    if (err != cudaSuccess) { // если ошибка
        cout << "CUDA error (" << msg << "): " << cudaGetErrorString(err) << "\n"; // выводит ошибку
        exit(1); // завершает
    }
}

__global__ void mul2_kernel(float* a, int n) { // kernel умножает на 2
    int i = blockIdx.x * blockDim.x + threadIdx.x; // индекс
    if (i < n) a[i] *= 2.0f; // умножает
}

static void cpu_mul2(float* a, int n) { // CPU умножение
    for (int i = 0; i < n; i++) a[i] *= 2.0f; // цикл
}

static float event_ms(cudaEvent_t s, cudaEvent_t e) { // считает время между событиями
    float ms = 0.0f; // ms
    cuda_ok(cudaEventElapsedTime(&ms, s, e), "elapsed"); // elapsed
    return ms; // ms
}

int main(int argc, char** argv) { // main
    int N = 10000000; // размер
    int chunks = 2; // чанки GPU
    if (argc >= 2) N = atoi(argv[1]); // читает N
    if (argc >= 3) chunks = atoi(argv[2]); // читает chunks

    int threads = 256; // потоки
    int half = N / 2; // первая половина CPU
    int n_gpu = N - half; // вторая половина GPU

    float* h = nullptr; // pinned host память
    cuda_ok(cudaMallocHost(&h, N * (int)sizeof(float)), "mallocHost"); // pinned alloc
    for (int i = 0; i < N; i++) h[i] = (float)(i % 100); // заполняет

    float* d = nullptr; // память GPU для второй половины
    cuda_ok(cudaMalloc(&d, n_gpu * (int)sizeof(float)), "cudaMalloc d"); // alloc

    cudaStream_t st1; // stream 1
    cudaStream_t st2; // stream 2
    cuda_ok(cudaStreamCreate(&st1), "stream create"); // create
    cuda_ok(cudaStreamCreate(&st2), "stream create"); // create

    cudaEvent_t e1; // событие start
    cudaEvent_t e2; // событие end
    cuda_ok(cudaEventCreate(&e1), "event create"); // create
    cuda_ok(cudaEventCreate(&e2), "event create"); // create

    cuda_ok(cudaEventRecord(e1, 0), "record"); // start h2d
    cuda_ok(cudaMemcpy(d, h + half, n_gpu * (int)sizeof(float), cudaMemcpyHostToDevice), "h2d"); // sync copy
    cuda_ok(cudaEventRecord(e2, 0), "record"); // end h2d
    cuda_ok(cudaEventSynchronize(e2), "sync"); // sync
    float h2d_only = event_ms(e1, e2); // время h2d

    int blocks_gpu = (n_gpu + threads - 1) / threads; // блоки
    cuda_ok(cudaEventRecord(e1, 0), "record"); // start kernel
    mul2_kernel<<<blocks_gpu, threads>>>(d, n_gpu); // kernel
    cuda_ok(cudaGetLastError(), "kernel"); // check
    cuda_ok(cudaEventRecord(e2, 0), "record"); // end kernel
    cuda_ok(cudaEventSynchronize(e2), "sync"); // sync
    float k_only = event_ms(e1, e2); // время kernel

    cuda_ok(cudaEventRecord(e1, 0), "record"); // start d2h
    cuda_ok(cudaMemcpy(h + half, d, n_gpu * (int)sizeof(float), cudaMemcpyDeviceToHost), "d2h"); // sync copy
    cuda_ok(cudaEventRecord(e2, 0), "record"); // end d2h
    cuda_ok(cudaEventSynchronize(e2), "sync"); // sync
    float d2h_only = event_ms(e1, e2); // время d2h

    double t0 = omp_get_wtime(); // wall start

#pragma omp parallel sections // две секции CPU и GPU
    {
#pragma omp section // GPU секция
        {
            int chunk = (n_gpu + chunks - 1) / chunks; // размер чанка
            for (int c = 0; c < chunks; c++) { // цикл чанков
                int off = c * chunk; // offset
                int cur = chunk; // текущий размер
                if (off + cur > n_gpu) cur = n_gpu - off; // обрезка
                if (cur <= 0) continue; // защита

                cudaStream_t st = (c % 2 == 0 ? st1 : st2); // выбирает стрим

                cuda_ok(cudaMemcpyAsync(d + off, h + half + off, cur * (int)sizeof(float), cudaMemcpyHostToDevice, st), "h2d async"); // async h2d

                int bl = (cur + threads - 1) / threads; // blocks
                mul2_kernel<<<bl, threads, 0, st>>>(d + off, cur); // kernel в этом stream

                cuda_ok(cudaMemcpyAsync(h + half + off, d + off, cur * (int)sizeof(float), cudaMemcpyDeviceToHost, st), "d2h async"); // async d2h
            }

            cuda_ok(cudaStreamSynchronize(st1), "sync st1"); // ждёт stream1
            cuda_ok(cudaStreamSynchronize(st2), "sync st2"); // ждёт stream2
        }

#pragma omp section // CPU секция
        {
            cpu_mul2(h, half); // CPU обрабатывает первую половину
        }
    }

    cuda_ok(cudaDeviceSynchronize(), "device sync"); // на всякий случай
    double t1 = omp_get_wtime(); // wall end
    double total_ms = (t1 - t0) * 1000.0; // total ms

    cout << "task 3\n"; // метка
    cout << "n = " << N << "\n"; // n
    cout << "chunks = " << chunks << "\n"; // chunks
    cout << "h2d only ms = " << h2d_only << "\n"; // h2d
    cout << "kernel only ms = " << k_only << "\n"; // kernel
    cout << "d2h only ms = " << d2h_only << "\n"; // d2h
    cout << "total hybrid ms = " << total_ms << "\n"; // total

    cuda_ok(cudaEventDestroy(e1), "event destroy"); // destroy
    cuda_ok(cudaEventDestroy(e2), "event destroy"); // destroy
    cuda_ok(cudaStreamDestroy(st1), "stream destroy"); // destroy
    cuda_ok(cudaStreamDestroy(st2), "stream destroy"); // destroy
    cuda_ok(cudaFree(d), "cudaFree"); // free
    cuda_ok(cudaFreeHost(h), "cudaFreeHost"); // free
    return 0; // exit
}


Overwriting task3_hybrid.cu


In [13]:
%%bash
nvcc -O2 -Xcompiler -fopenmp -gencode arch=compute_75,code=sm_75 task3_hybrid.cu -o task3_hybrid
./task3_hybrid 10000000 1
./task3_hybrid 10000000 2
./task3_hybrid 10000000 4


task 3
n = 10000000
chunks = 1
h2d only ms = 1.66934
kernel only ms = 0.352736
d2h only ms = 1.55309
total hybrid ms = 11.4796
task 3
n = 10000000
chunks = 2
h2d only ms = 1.65635
kernel only ms = 0.296736
d2h only ms = 1.54227
total hybrid ms = 4.1324
task 3
n = 10000000
chunks = 4
h2d only ms = 1.65587
kernel only ms = 0.295168
d2h only ms = 1.54262
total hybrid ms = 4.08156


In [15]:
%%writefile task4_mpi.cpp

#include <mpi.h> // подключает MPI
#include <iostream> // подключает cout
#include <vector> // подключает vector
#include <cstdlib> // подключает atoi
#include <climits> // подключает INT_MAX INT_MIN

using namespace std; // чтобы не писать std::

static void fill_array(vector<int>& a) { // заполняет массив
    srand(123); // seed
    for (int i = 0; i < (int)a.size(); i++) { // цикл
        a[i] = rand() % 1000; // кладёт 0..999
    }
}

int main(int argc, char** argv) { // main
    MPI_Init(&argc, &argv); // init MPI

    int rank = 0; // rank
    int size = 1; // size
    MPI_Comm_rank(MPI_COMM_WORLD, &rank); // rank
    MPI_Comm_size(MPI_COMM_WORLD, &size); // size

    int mode = 0; // 0 strong, 1 weak
    int Nstrong = 20000000; // N для strong
    int Npp = 5000000; // N на процесс для weak
    if (argc >= 2) mode = atoi(argv[1]); // mode
    if (argc >= 3) Nstrong = atoi(argv[2]); // Nstrong
    if (argc >= 4) Npp = atoi(argv[3]); // Npp

    int globalN = (mode == 0 ? Nstrong : Npp * size); // итоговый N

    vector<int> full; // полный массив
    if (rank == 0) { // только root
        full.assign(globalN, 0); // alloc
        fill_array(full); // fill
    }

    vector<int> counts(size, 0); // counts
    vector<int> displs(size, 0); // displs
    int base = globalN / size; // база
    int rem = globalN % size; // остаток
    for (int r = 0; r < size; r++) { // цикл по ранкам
        counts[r] = base + (r < rem ? 1 : 0); // раздаёт остаток
    }
    displs[0] = 0; // первое смещение
    for (int r = 1; r < size; r++) { // цикл смещений
        displs[r] = displs[r - 1] + counts[r - 1]; // суммирует
    }

    int local_n = counts[rank]; // локальный размер
    vector<int> local(local_n); // локальный массив

    MPI_Barrier(MPI_COMM_WORLD); // барьер
    double t0 = MPI_Wtime(); // start time

    MPI_Scatterv( // scatterv
        rank == 0 ? full.data() : nullptr, // sendbuf
        counts.data(), // sendcounts
        displs.data(), // displs
        MPI_INT, // тип
        local.data(), // recvbuf
        local_n, // recvcount
        MPI_INT, // тип
        0, // root
        MPI_COMM_WORLD // comm
    );

    long long local_sum = 0; // local sum
    int local_min = INT_MAX; // local min
    int local_max = INT_MIN; // local max
    for (int i = 0; i < local_n; i++) { // цикл
        int x = local[i]; // x
        local_sum += x; // sum
        if (x < local_min) local_min = x; // min
        if (x > local_max) local_max = x; // max
    }

    long long sum_reduce = 0; // reduce sum
    int min_reduce = 0; // reduce min
    int max_reduce = 0; // reduce max

    MPI_Reduce(&local_sum, &sum_reduce, 1, MPI_LONG_LONG, MPI_SUM, 0, MPI_COMM_WORLD); // reduce sum
    MPI_Reduce(&local_min, &min_reduce, 1, MPI_INT, MPI_MIN, 0, MPI_COMM_WORLD); // reduce min
    MPI_Reduce(&local_max, &max_reduce, 1, MPI_INT, MPI_MAX, 0, MPI_COMM_WORLD); // reduce max

    double t1 = MPI_Wtime(); // end time

    long long sum_all = local_sum; // allreduce sum
    int min_all = local_min; // allreduce min
    int max_all = local_max; // allreduce max

    MPI_Allreduce(MPI_IN_PLACE, &sum_all, 1, MPI_LONG_LONG, MPI_SUM, MPI_COMM_WORLD); // allreduce sum
    MPI_Allreduce(MPI_IN_PLACE, &min_all, 1, MPI_INT, MPI_MIN, MPI_COMM_WORLD); // allreduce min
    MPI_Allreduce(MPI_IN_PLACE, &max_all, 1, MPI_INT, MPI_MAX, MPI_COMM_WORLD); // allreduce max

    if (rank == 0) { // печатает только root
        cout << "task 4\n"; // метка
        cout << "mode = " << (mode == 0 ? "strong" : "weak") << "\n"; // режим
        cout << "n = " << globalN << "\n"; // n
        cout << "np = " << size << "\n"; // np
        cout << "time reduce sec = " << (t1 - t0) << "\n"; // время reduce
        cout << "sum = " << sum_reduce << "\n"; // sum
        cout << "min = " << min_reduce << "\n"; // min
        cout << "max = " << max_reduce << "\n"; // max
        cout << "allreduce sum = " << sum_all << "\n"; // allreduce sum
        cout << "allreduce min = " << min_all << "\n"; // allreduce min
        cout << "allreduce max = " << max_all << "\n"; // allreduce max
    }

    MPI_Finalize(); // finalize
    return 0; // exit
}


Overwriting task4_mpi.cpp


In [17]:
%%bash
mpic++ -O2 task4_mpi.cpp -o task4_mpi

# strong scaling (N фикс)
mpirun --allow-run-as-root --oversubscribe -np 1 ./task4_mpi 0 20000000
mpirun --allow-run-as-root --oversubscribe -np 2 ./task4_mpi 0 20000000
mpirun --allow-run-as-root --oversubscribe -np 4 ./task4_mpi 0 20000000
mpirun --allow-run-as-root --oversubscribe -np 8 ./task4_mpi 0 20000000

# weak scaling (N растёт с np)
mpirun --allow-run-as-root --oversubscribe -np 1 ./task4_mpi 1 0 5000000
mpirun --allow-run-as-root --oversubscribe -np 2 ./task4_mpi 1 0 5000000
mpirun --allow-run-as-root --oversubscribe -np 4 ./task4_mpi 1 0 5000000
mpirun --allow-run-as-root --oversubscribe -np 8 ./task4_mpi 1 0 5000000


task 4
mode = strong
n = 20000000
np = 1
time reduce sec = 0.0380552
sum = 9989562151
min = 0
max = 999
allreduce sum = 9989562151
allreduce min = 0
allreduce max = 999
